In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import pandas as pd
from datasets import load_dataset

In [ ]:
data_files = {"train": "dataset_final_train_val.csv", "test": "Dataset_final-test.csv"}
df = load_dataset("csv", data_files=data_files)

checkpoint = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["content"],truncation=True)


tokenized_datasets = df.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
df

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Unnamed: 0", "tweetid", "content"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

In [ ]:
tokenized_datasets['train'][0]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
#eval_dataloader = DataLoader(
#    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
#)
eval_test= DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
#Para crear la devaluacion del Learning rate
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
## Para ver la progresion durante el entrenamiento
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
#Metrica para medir resultados obtenidos por batches
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_test:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## Prueba con Tweets Bitcoin

In [ ]:
# abrimos el dataset
#data_files = {"test_bitcoin": "Prueba dataset bitcoin.csv"}
data_bit = load_dataset("csv", data_files={"test_bitcoin": "cleanTweets.csv"})
data_bit

In [ ]:
tokenized_data_test = data_bit.map(tokenize_function, batched=True)

In [ ]:
tokenized_data_test

In [ ]:
tokenized_data_test = tokenized_data_test.remove_columns([ 'Unnamed: 0',"id", 'date',"content"])
tokenized_data_test.set_format("torch")
tokenized_data_test["test_bitcoin"].column_names

In [ ]:
test_bitloader = DataLoader(
    tokenized_data_test["test_bitcoin"], shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
etiquetas=[]
model.eval()
for batch in test_bitloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    etiquetas.extend(predictions.tolist())


In [ ]:
data_bit.set_format("pandas")

In [ ]:
data_bit

In [ ]:
d_bit=data_bit['test_bitcoin'][:]

In [ ]:
d_bit['label']=etiquetas

In [ ]:
d_bit['label'].replace({ 0 : 'N', 1: "NEU", 2: "P"}, inplace=True)

In [ ]:
from datetime import datetime

In [ ]:
fechas=[]
for date in d_bit['date']:
    fechas.append((datetime.strptime(date, '%d/%m/%Y %H:%M')).strftime('%Y-%m'))

In [ ]:
d_bit['fechas']=fechas
d_bit.to_csv("TweetsEtiquetados-año.csv")

In [ ]:
d_bit['label']